In [14]:
# imports
from datasets import load_dataset, Dataset
import os, random
import numpy as np
import torch, transformers
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    Trainer,
    TrainingArguments,
)
from trl import DPOTrainer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import mlflow
from peft import PeftModel

ModuleNotFoundError: No module named 'trl'

In [ ]:
# load environment variables
def get_env(text_file: bool = False):
    path = "/workspace/envars.txt" if text_file else "/workspace/.env"
    cfg = {}

    with open(path, "r") as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith("#"):
                continue
            key, value = line.split("=", 1)
            cfg[key.strip()] = value.strip()

    return cfg


envars = get_env(text_file=False)


def envar(var: str, dtype: str = "str"):
    if dtype == "int":
        return int(envars.get(var.upper()))
    elif dtype == "float":
        return float(envars.get(var.upper()))
    elif dtype == "bool":
        return envars.get(var.upper()).strip().lower() in {"1", "true", "yes", "y"}
    elif dtype == "str":
        return envars.get(var.upper())

In [ ]:
# stage 0: safety check

## device
device = torch.device(
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

## cuda
cuda_available = torch.cuda.is_available()

print(f"device: {device} | cuda: {cuda_available}")

if cuda_available:
    gpu_count = torch.cuda.device_count()
    gpu_list = [torch.cuda.get_device_name(i) for i in range(gpu_count)]

    print(f"{gpu_count} GPUs available: {gpu_list}")


## seed
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


set_seed(envar("SEED", "int"))
print(f"seed: {envar('SEED', 'int')}")

## allow tf32

torch.backends.cuda.matmul.allow_tf32 = True


device: cuda | cuda: True
1 GPUs available: ['NVIDIA A40']
seed: 42


In [ ]:
# stage 1: load pref data
def convert_pref_record(record):
    prompt = record.get("prompt")
    chosen = record.get("chosen")
    rejected = record.get("rejected")
    if prompt and chosen and rejected:
        return {"prompt": prompt, "chosen": chosen, "rejected": rejected}
    return None

raw_pref = load_dataset(
    envar("pref_dataset"), split=f"train_prefs[:{envar('pref_sample_size')}]"
)

pref_cleaned = [convert_pref_record(r) for r in raw_pref if convert_pref_record(r) is not None]

pref_dataset = Dataset.from_dict({
    "prompt": [record["prompt"] for record in pref_cleaned],
    "chosen": [record["chosen"] for record in pref_cleaned],
    "rejected": [record["rejected"] for record in pref_cleaned],
})

20000


In [ ]:
# stage 2: training configs
## BitsAndBytes (QLoRA) config
qlora_config = BitsAndBytesConfig(
    load_in_4bit=envar("load_in_4bit", "bool"),
    bnb_4bit_compute_dtype={"bfloat16": torch.bfloat16, "float16": torch.float16}[
        envar("bnb_compute_dtype")
    ],
    bnb_4bit_quant_type=envar("bnb_quant_type"),
    bnb_4bit_use_double_quant=envar("bnb_double_quant", "bool"),
)

## PEFT config
# peft_config = LoraConfig(
#     r=envar("lora_r", "int"),
#     lora_alpha=envar("lora_alpha", "int"),
#     lora_dropout=envar("lora_dropout", "float"),
#     target_modules=[
#         module.strip()
#         for module in envar("target_modules").split(",")
#         if module.strip()
#     ],
#     bias="none",
#     task_type="CAUSAL_LM",
#     use_dora=envar("use_dora", "bool"),
# )

## TrainingArguments
training_args = TrainingArguments(
    output_dir=os.path.join(envar("save_dir"), "pref"),
    per_device_train_batch_size=envar("per_device_train_batch_size", "int"),
    gradient_accumulation_steps=envar("gradient_accumulation_steps", "int"),
    learning_rate=envar("learning_rate", "float") * 0.5,
    warmup_ratio=envar("warmup_ratio", "float"),
    weight_decay=envar("weight_decay", "float"),
    num_train_epochs=envar("num_train_epochs", "int"),
    logging_steps=25,
    save_strategy="epoch",
    bf16=(envar("bnb_compute_dtype") == "bfloat16"),
    gradient_checkpointing=False,
    max_grad_norm=envar("grad_clip", "float"),
    report_to="mlflow",
    run_name="ultrahermes-dpo",
    save_total_limit=2,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    dataloader_num_workers=4,
    dataloader_pin_memory=True,
    ddp_find_unused_parameters=False,
)

In [ ]:
# stage 3: load tokenizer, policy and reference models

## 1. load tokenizer
tokenizer = AutoTokenizer.from_pretrained(envar("save_dir"))

if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

## 2. load model
def load_model():
    base_model = AutoModelForCausalLM.from_pretrained(
        envar("model_id"),
        device_map="auto",
        quantization_config=qlora_config,
        attn_implementation=envar("attn_implementation"),
    )
    return PeftModel.from_pretrained(base_model, envar("save_dir"))
    
### policy model preparation
policy_model = load_model()
policy_model.config.use_cache = False
policy_model.config.pad_token_id = tokenizer.pad_token_id

print("policy model loaded")

### reference model preparation
reference_model = load_model()
for param in reference_model.parameters():
    param.requires_grad = False
reference_model.eval()

print("reference model loaded")

In [ ]:
# stage 4: tokenize and pack sft data
def build_packed_dataset(token_stream, eos_id, max_token_length):
    all_ids = []
    for ids in token_stream["input_ids"]:
        if not ids:
            continue
        all_ids.extend(ids)
        if eos_id is not None:
            all_ids.append(eos_id)
            
    chunks = [all_ids[i:i+max_token_length] for i in range(0, len(all_ids) - max_token_length + 1, max_token_length)]
    print(f"tokens: {len(all_ids):,} | {len(chunks):,} chunks of {max_token_length} tokens each")
    return Dataset.from_dict({"input_ids": chunks})

max_token_length = envar("max_seq_len", "int")
eos_id = tokenizer.eos_token_id

## tokenize without truncation (no padding or special tokens)
def tokenize(batch):
    return tokenizer(batch["text"], add_special_tokens=False, truncation=False)

sft_stream = sft_dataset.map(tokenize, batched=True, remove_columns=["text"])

## pack data
sft_packed = build_packed_dataset(sft_stream, eos_id, max_token_length)
sft_packed = sft_packed.map(lambda ex: {
    "attention_mask": [1]*len(ex["input_ids"]),
    "labels": ex["input_ids"]
})

In [ ]:
# stage 5: build trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=sft_packed,
    data_collator=None,
)

In [ ]:
# stage 6: train
trainer.train()

In [ ]:
# stage 7: save model
model.save_pretrained(envar("save_dir"))
tokenizer.save_pretrained(envar("save_dir"))